In this exercise, you will perform prompt engineering on a dialogue summarization task using [Flan-T5](https://huggingface.co/google/flan-t5-large) and the [dialogsum dataset](https://huggingface.co/datasets/knkarthick/dialogsum). You will explore how different prompts affect the output of the model, and compare zero-shot and few-shot inferences. <br/>
Complete the code in the cells below.

#Manish Kanuri
#002315456

### 1. Set up Required Dependencies

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
from datasets import load_dataset

### 2. Explore the Dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset('knkarthick/dialogsum')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Print several dialogues with their baseline summaries.

In [4]:
example_indices = [0, 42, 800]
dash_line = '-' * 100

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

----------------------------------------------------------------------------------------------------
Example 1
----------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to 

### 3. Summarize Dialogues without Prompt Engineering

Load the Flan-T5-large model and its tokenizer.

In [5]:
model_name = 'google/flan-t5-large'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

**Exercise**: Use the pre-trained model to summarize the example dialogues without any prompt engineering. Use the `model.generate()` function with `max_new_tokens=50`.

In [6]:
### WRITE YOUR CODE HERE

from transformers import pipeline

summarizer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

for i in example_indices:
    input_text = dataset['test'][i]['dialogue']
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=50)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n[Dialog {i} Summary]\n{summary}\n")


Device set to use cpu



[Dialog 0 Summary]
#Person1: Ms. Dawson, please take dictation for me.


[Dialog 42 Summary]
#Person1#: Thank you, #Person2#.


[Dialog 800 Summary]
#Person1#: That's great.



You can see that the model generations make some sense, but the model doesn't seem to be sure what task it is supposed to accomplish and it often just makes up the next sentence in the dialogue. Prompt engineering can help here.

### 4. Summarize Dialogues with Instruction Prompts

In order to instruct the model to perform a task (e.g., summarize a dialogue), you can take the dialogue and convert it into an instruction prompt. This is often called **zero-shot inference**.

**Exercise**: Wrap the dialogues in a descriptive instruction (e.g., "Summarize the following conversation."), and examine how the generated text changes.

In [7]:
### WRITE YOUR CODE HERE
for i in example_indices:
    prompt = f"Summarize the following conversation:\n{dataset['test'][i]['dialogue']}\nSummary:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=50)
    print(f"\n[Dialog {i} Summary]\n{tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")




[Dialog 0 Summary]
#Person1: I need you to take dictation for me. #Person2: Yes, sir. #Person1: All office communications are restricted to email correspondence and official memos. The use of Instant Message


[Dialog 42 Summary]
#Person1# is worried about his future. #Person2# gives him some advice.


[Dialog 800 Summary]
Dad, you keep talking about your uncle Bill, his wife and two of their daughters in New Zealand.



This is much better! But the model still does not pick up on the nuance of the conversations though.

**Exercise:** Experiment with the prompt text and see how it influences the generated output. Do the inferences change if you end the prompt with just empty string vs. `Summary: `?

In [9]:
### WRITE YOUR CODE HERE
# Choose a dialogue index to test
example_index = 0
dialogue = dataset['test'][example_index]['dialogue']
human_summary = dataset['test'][example_index]['summary']

# Prompt A: No suffix
prompt_a = f"Summarize the following conversation:\n{dialogue}"
inputs_a = tokenizer(prompt_a, return_tensors="pt", truncation=True)
output_a = model.generate(**inputs_a, max_new_tokens=50)
summary_a = tokenizer.decode(output_a[0], skip_special_tokens=True)

# Prompt B: Ends with 'Summary:'
prompt_b = f"Summarize the following conversation:\n{dialogue}\nSummary:"
inputs_b = tokenizer(prompt_b, return_tensors="pt", truncation=True)
output_b = model.generate(**inputs_b, max_new_tokens=50)
summary_b = tokenizer.decode(output_b[0], skip_special_tokens=True)

# Print comparisons
print("=== Human Summary ===")
print(human_summary)
print("\n=== Prompt A (no suffix) ===")
print(summary_a)
print("\n=== Prompt B (with 'Summary:') ===")
print(summary_b)



=== Human Summary ===
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.

=== Prompt A (no suffix) ===
#Person1# wants Ms. Dawson to take dictation for him.

=== Prompt B (with 'Summary:') ===
#Person1: I need you to take dictation for me. #Person2: Yes, sir. #Person1: All office communications are restricted to email correspondence and official memos. The use of Instant Message


**Exercise:** Flan-T5 has many prompt templates that are published for certain tasks [here](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py). Try using its pre-built prompts for dialogue summarization (e.g., the ones under the `"samsum"` key) and see how they influence the outputs.


In [15]:
# Simulate a Flan-T5 samsum-style prompt manually (as huggingface doesn't expose it directly)
index = 42  # or any test index you'd like
dialogue = dataset['test'][index]['dialogue']

prompt = f"Dialog:\n{dialogue}\nTL;DR:"

inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs, max_new_tokens=50)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== Pre-Built Style Prompt Summary ===")
print(summary)
print("\n=== Human Summary ===")
print(dataset['test'][index]['summary'])


=== Pre-Built Style Prompt Summary ===
Person1 is worried about her future.

=== Human Summary ===
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.


Notice that the prompts from Flan-T5 did help, but the model still struggles to pick up on the nuance of the conversation in some cases. This is what you will try to solve with few-shot inferencing.

### 5. Summarize Dialogues with a Few-Shot Inference

**Few-shot inference** is the practice of providing an LLM with several examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task.

**Exercise:** Build a function that takes a list of `in_context_example_indexes`, generates a prompt with the examples, then at the end appends the prompt that you want the model to complete (`test_example_index`). Use the same Flan-T5 prompt template from Section 3. Make sure to separate between the examples with `"\n\n\n"`.

In [10]:
def make_prompt(in_context_example_indices, test_example_index):
    prompt = ""
    for idx in in_context_example_indices:
        dialogue = dataset['test'][idx]['dialogue']
        summary = dataset['test'][idx]['summary']
        prompt += f"Summarize the following conversation:\n{dialogue}\nSummary: {summary}\n\n\n"

    test_dialogue = dataset['test'][test_example_index]['dialogue']
    prompt += f"Summarize the following conversation:\n{test_dialogue}\nSummary:"
    return prompt


In [11]:
in_context_example_indices = [0, 10, 20]
test_example_index = 800

few_shot_prompt = make_prompt(in_context_example_indices, test_example_index)
print(few_shot_prompt)

Summarize the following conversation:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! 

Now pass this prompt to the model perform a few shot inference:

In [12]:
### WRITE YOUR CODE HERE
inputs = tokenizer(few_shot_prompt, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Summary: It's Brian's birthday. He's going to have a dance with Person2.


**Exercise:** Experiment with the few-shot inferencing:
- Choose different dialogues - change the indices in the `in_context_example_indices` list and `test_example_index` value.
- Change the number of examples. Be sure to stay within the model's 512 context length, however.

How well does few-shot inference work with other examples?

In [13]:
### WRITE YOUR CODE HERE
# Select a few different examples for few-shot context and one for testing
in_context_example_indices = [10, 25, 50]  # You can change these
test_example_index = 42                   # And change this one too

def make_prompt(in_context_example_indices, test_example_index):
    prompt = ""
    for idx in in_context_example_indices:
        dialogue = dataset['test'][idx]['dialogue']
        summary = dataset['test'][idx]['summary']
        prompt += f"Summarize the following conversation:\n{dialogue}\nSummary: {summary}\n\n\n"

    test_dialogue = dataset['test'][test_example_index]['dialogue']
    prompt += f"Summarize the following conversation:\n{test_dialogue}\nSummary:"
    return prompt

# Generate the prompt
few_shot_prompt = make_prompt(in_context_example_indices, test_example_index)

# Tokenize and generate the summary
inputs = tokenizer(few_shot_prompt, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs, max_new_tokens=50)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print result
print("=== FEW-SHOT GENERATED SUMMARY ===")
print(summary)
print("\n=== ACTUAL HUMAN SUMMARY ===")
print(dataset['test'][test_example_index]['summary'])



=== FEW-SHOT GENERATED SUMMARY ===
Summary: Person1 is going to make a stripe.

=== ACTUAL HUMAN SUMMARY ===
#Person1# wants to adjust #Person1#'s life and #Person2# suggests #Person1# be positive and stay healthy.


### 6. Generative Configuration Parameters for Inference

You can change the configuration parameters of the `generate()` method to see a different output from the LLM. So far the only parameter that you have been setting was `max_new_tokens=50`, which defines the maximum number of tokens to generate. A convenient way of organizing the configuration parameters is to use `GenerationConfig` class. By setting the parameter `do_sample = True`, you can activate various decoding strategies which influence the next token from the probability distribution over the entire vocabulary. You can then adjust the outputs changing `temperature` and other parameters (such as `top_k` and `top_p`). A full list of available parameters can be found in the [Hugging Face Generation documentation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig).

**Exercise:** Change the configuration parameters to investigate their influence on the output. Analyze your results.

In [16]:
from transformers import GenerationConfig

dialogue = dataset['test'][0]['dialogue']
prompt = f"Summarize the following conversation:\n{dialogue}\nSummary:"

inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

# Try a creative decoding setup
gen_config = GenerationConfig(
    max_new_tokens=50,
    do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95
)

outputs = model.generate(**inputs, generation_config=gen_config)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== Sampled Summary (temperature=0.9, top_k=50, top_p=0.95) ===")
print(summary)
print("\n=== Human Summary ===")
print(dataset['test'][0]['summary'])


=== Sampled Summary (temperature=0.9, top_k=50, top_p=0.95) ===
#Person1 is going to give an official memo to all employees. #Person2 will type up the memo and send it out to all employees.

=== Human Summary ===
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


In [14]:
### WRITE YOUR CODE HERE
from transformers import GenerationConfig

config = GenerationConfig(
    max_new_tokens=50,
    do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95
)

prompt = f"Summarize the following conversation:\n{dataset['test'][0]['dialogue']}\nSummary:"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs, generation_config=config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



#Person1# wants Ms. Dawson to type up and distribute a memo to all employees on new communications policy. She should send it out before 4 pm.
